In [ ]:
import os
import cv2
import sys
%cd  ..
%cd yolov7/
os.getcwd()
# !pip install -r requirements.txt -q           #use this for the if running this file for the first time
os.getcwd()

In [ ]:
"""video_path = '../test_videos/v1.mp4'
!python ./yolov7/detect.py --weights ./best.pt --conf 0.25 --img-size 640 --source ./test_videos/v1.mp4"""

In [ ]:
import torch
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# Assuming you have a detect function defined somewhere

In [ ]:
# Function to calculate bounding box coordinates from YOLO output

def get_BBOX_coordinates(image_width, image_height, x_center, y_center, width, height):
    # Calculate pixel coordinates
    x_center_pixel = x_center * image_width
    y_center_pixel = y_center * image_height
    half_width = (width * image_width) / 2
    half_height = (height * image_height) / 2

    # Calculate top-left and bottom-right coordinates
    x_min = int(x_center_pixel - half_width)
    y_min = int(y_center_pixel - half_height)
    x_max = int(x_center_pixel + half_width)
    y_max = int(y_center_pixel + half_height)

    return x_min, y_min, x_max, y_max


In [ ]:
import numpy as np



In [ ]:
import cv2
import torch
import numpy as np
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
#sys.path.append(parent_dir)
#from sounds import play_alert_sound
# Path to your custom weights
weights_path = '../best.pt'

# Load the YOLOv7 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = attempt_load(weights_path, map_location=device)
model.eval()

# Define the video path
video_path = '../test_videos/v1.mp4'

# Load the video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Cannot open video file {video_path}")
    exit()

# Define polygon coordinates
polygon_points = np.array([(380, 100), (380, 590), (480, 100), (480, 590)], np.int32)

# Loop through the video frames
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Reached the end of the video or failed to read frame.")
        break

    # Pre-process the frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(frame_rgb, (640, 480))
    img = img.transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    img_tensor = torch.from_numpy(img).to(device).float() / 255.0
    if img_tensor.ndimension() == 3:
        img_tensor = img_tensor.unsqueeze(0)

    # Run inference
    with torch.no_grad():
        pred = model(img_tensor)[0]

    # Apply NMS
    pred = non_max_suppression(pred, 0.5, 0.45, classes=None, agnostic=False)

    # Check bounding boxes against the polygon
    for det in pred:
        if det is not None and len(det):
            det[:, :4] = scale_coords(img_tensor.shape[2:], det[:, :4], frame.shape).round()

            for *xyxy, conf, cls in det:
                x1, y1, x2, y2 = map(int, xyxy)

                # Check corners of the bounding box
                corners = [(x1, y1), (x2, y1), (x1, y2), (x2, y2)]
                inside_polygon = any(cv2.pointPolygonTest(polygon_points, corner, False) >= 0 for corner in corners)

                if inside_polygon:
                    cv2.putText(frame, "Alert!", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display the annotated frame
    cv2.imshow('Annotated Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
#remember always stay in yolov7 dir and not in its parent dir other wise no modula named models error will pop 